# Preparació de dades

En aquest *notebook*, convertim les dades originals a formats que es poden treballar mès fàcilment a *Python* i vam mostrar petits anàlisi - amb l'esperança que serveixin d'inspiració.

In [1]:
import pandas as pd
import numpy as np 
import plydata as ply
import datetime
import yaml
from functools import lru_cache

ply.options.modify_input_data = True


### Pas 1: llegir arxiu

In [2]:
original = pd.read_csv("data/input/20171101_actualitzat_llistat_Llei_11_2017.csv",delimiter=";",encoding="latin-1")

### Un primer cop d'ull a les dades

In [3]:
# sexe "--" marca persones jurídiques

original.query("sexe != '--'").head()

,Codi,cognoms nom,cognoms,nom,sexe,edat,Municipi naixement,Pedanies/Agregats naixement,Comarca naixement,Província naixement,...,Tipus procediment 2,num causa,any inicial,any aprovació sen o altra resol,pena,commutació/indult(demanat),afusellades,ref num arxiu,autoria i data de la descripció,data correccio registre
17,135218,"ABAD ALFONSO, Erundina",ABAD ALFONSO,Erundina,Dona,43.0,Alcoi,--,Alcoià,Alacant,...,Sumaríssim,030827,1942.0,1943.0,Sobreseïment,NaN,NaN,37729.0,ANC 2017 07 12,NaN
18,25335,"ABAD ARBÓS, Ricardo",ABAD ARBÓS,Ricardo,Home,30.0,Barcelona,--,Barcelonès,Barcelona,...,--,--,1939.0,1939.0,Sense declaració de responsabilitats,NaN,NaN,12842.0,ANC 2017 07 12,NaN
19,123192,"ABAD BARAS, José",ABAD BARAS,José,Home,30.0,Benavarri,--,Ribagorça,Osca,...,Sumaríssim,000940,1939.0,1939.0,Absolt,NaN,NaN,49476.0,ANC 2017 07 12,NaN
20,171,"ABAD BATLLONE, Juan",ABAD BATLLONE,Juan,Home,42.0,Barcelona,--,Barcelonès,Barcelona,...,Diligències prèvies,--,1939.0,1939.0,Sense declaració de responsabilitats,NaN,NaN,4063.0,ANC 2017 07 12,NaN
21,24485,"ABAD BOIRA, Ricardo",ABAD BOIRA,Ricardo,Home,48.0,Tauste,--,--,Saragossa,...,Diligències prèvies,--,1941.0,1942.0,Sense declaració de responsabilitats,NaN,NaN,12915.0,ANC 2017 07 12,NaN


### Pas 2: Canviar noms de columnes a identificadors permesos en *Python*

In [4]:
original >> ply.rename(
{'afusellades': 'afusellades',
 'any_inicial': 'any inicial',
 'any_resol': 'any aprovació sen o altra resol',
 'ca_naix': 'Comunitat autònoma naixement',
 'ca_res': 'Comunitat autònoma residència',
 'codi': 'Codi',
 'cognoms': 'cognoms',
 'cognoms_nom': 'cognoms nom',
 'com_naix': 'Comarca naixement',
 'com_res': 'Comarca residència',
 'correccio': 'data correccio registre',
 'descr': 'autoria i data de la descripció',
 'edat': 'edat',
 'indult': 'commutació/indult(demanat)',
 'mun_naix': 'Municipi naixement',
 'mun_res': 'Municipi residència',
 'nom': 'nom',
 'num_causa': 'num causa',
 'pais_naix': 'País naixement',
 'pais_res': 'País residència',
 'ped_naix': 'Pedanies/Agregats naixement',
 'ped_res': 'Pedanies/Agregats residencia',
 'pena': 'pena',
 'proc_1': 'Tipus procediment 1',
 'proc_2': 'Tipus procediment 2',
 'prov_naix': 'Província naixement',
 'prov_res': 'Província residència',
 'ref_arxiu': 'ref num arxiu',
 'sexe': 'sexe'}
)

pass


### Pas 3: Canviar números d'anys a dates per automatitzar el càlcul dels eixos 


In [5]:
# l'ús de lru_cache fa tot més ràpid, perquè cada any només ha de ser convertit a data una sola vegada


@lru_cache(maxsize=100)
def pd_year(n: float) :
    try :
        return pd.date_range(start=datetime.datetime(int(n),1,1),end=datetime.datetime(int(n),12,31),freq="Y")[0]
    except (TypeError,ValueError,OverflowError) :
        return n


original["any_inicial"] = original["any_inicial"].apply(pd_year)
original["any_resol"] = original["any_resol"].apply(pd_year)


### Primera anàlisi: els sumaríssims de més durada

In [6]:
dura = original.loc[:,["cognoms_nom","any_inicial","any_resol","sexe","pena","pais_naix"]]

dura["dura"] = dura["any_resol"] - dura["any_inicial"]

dura.sort_values(["dura"],ascending=False).iloc[:5]

,cognoms_nom,any_inicial,any_resol,sexe,pena,pais_naix,dura
55258,"ROIG ALTADILL, Ramón",1942-12-31,1974-12-31,Home,Arxiu,Espanya,11688 days
3172,"ARCELUS MARTÍNEZ, Aurelio",1942-12-31,1972-12-31,Home,Sobreseïment,Espanya,10958 days
17551,"COT REVERTER, Juan",1940-12-31,1969-12-31,Home,Repatriació,Espanya,10592 days
19057,"DOLZ BES, José María",1938-12-31,1966-12-31,Home,Sobreseïment,Espanya,10227 days
49033,"PIÑOL TORT, León",1941-12-31,1968-12-31,Home,Sobreseïment,Espanya,9862 days


### Pas 4: Categoritzar penes

Aquestes categories són derivades del [*Anexo de la Circular de Creación de las Comisiones de Examen de Penas* del any 1940](https://www.boe.es/datos/pdfs/BOE//1940/026/A00662-00665.pd). 

En alguns casos, les dades publicades són ambigus - per exemple quan la pena és de *"Sobreseïment provisional, tres mesos d'arrest major i llibertat"*. En aquests casos, categoritzem com la pena més severa esmentada (*"tres mesos"*). Si queden dubtes, la categoria es "altres" - per exemple amb *"Pena de multa, remissió o a disposició d'altres autoritats"*.

Un altre inexactitud: la etiqueta "sobreseïment" va ser usada no només per sobreseïment, sinó també per amagar tot tipo de morts en captivitat.

La relació de les categories amb les entrades a les dades publicats estan documentades a l'arxiu [categories.yml](https://github.com/martinvirtel/sumarissims-dades/blob/master/work/data/input/categories.yml). 

| categoria | descripció | |
| ----------| ----------- | --- |
|mort	|pena de mort	||	
|30a+	|30 anys i un dia a perpètua||		
|20-30a	|20 anys i un dia a 30 anys |Reclusión	Mayor|
|12-20a	|12 anys i un día a 20 anys|Menor|
|6-12a	|6 anys i un dia a 12 anys |Prisión	Mayor|
|6m-6a	|6 mesos i un dia a 6 anys		|Menor|
|<6m	|fins 6 mesos	||
|sob	|sobreseïment	||
|lib	|sin pena		||
|alt	|altres		||



In [7]:
penes_cat = yaml.load(open("data/input/categories.yml"))
ord_categories = { a[1]["nom"] : "%02d" % a[0] for a in enumerate(penes_cat["categories"]) }

categories_penes={ a["pena"] : "%s %s" % (ord_categories[a["categoria"]],a["categoria"])  
                   for a in penes_cat["etiquetes"] }

original["pena_cat"] = original["pena"].apply(
    lambda a: categories_penes.get(a,None))

### Primera anàlisi: relació de penes i sexe

Entre els homes, el grup més gran és el de condemnat a 12 - 20 anys (30.0%). Entre les dones, són les que queden en llibertat (40.8%).

In [8]:
penes = original.query("sexe != '--'").\
         groupby(["sexe","pena_cat"])["pena_cat"].count().\
         unstack("sexe")
        
penes["Dones %"] = penes["Dona"].div(sum(penes["Dona"])).apply(lambda a: "%.1f%%" % (a*100))
penes["Homes %"] = penes["Home"].div(sum(penes["Home"])).apply(lambda a: "%.1f%%" % (a*100))

penes

sexe,Dona,Home,Dones %,Homes %
pena_cat,,,,
00 mort,41,4376,0.7%,6.9%
01 30a+,119,3628,2.2%,5.7%
02 20-30a,50,1155,0.9%,1.8%
03 12-20a,856,18935,15.6%,30.0%
04 6-12a,590,2378,10.7%,3.8%
05 6m-6a,95,1673,1.7%,2.6%
06 <6m,247,766,4.5%,1.2%
07 sob,813,9382,14.8%,14.8%
08 lib,2246,14404,40.8%,22.8%


### Pas 5: Categoritzar origen  i residència

Categories: 

  - b - Barcelona
  - c - Catalunya (fora da província de Barcelona)
  - e - Espanya (fora de Catalunya)
  - i - internacional (fora de Espanya)
  - ? - altres / no consta
  


In [9]:
def cat_naix(cas) :
    if cas.prov_naix == '--' and cas.pais_naix == '--' :
         naix="?"
    if cas.prov_naix == "Barcelona" :
         naix="b"
    elif cas.ca_naix == "Catalunya" :
        naix="c"
    elif cas.pais_naix == "Espanya" :
        naix="e"
    else:
        naix="i"
    return naix
 
def cat_res(cas) :
    if cas.prov_res == '--' and cas.pais_res == '--' :
        return "?"
    if cas.prov_res == "Barcelona" :
        res="b"
    elif cas.ca_res == "Catalunya" :
        res="c"
    elif cas.pais_res.find("Espanya")>-1 :
        res="e"
    elif cas.pais_res.find("Sense domicili fix")>-1 :
        res="e"
    else :
        res="i"
    return res


original["cat_naix"]=original.apply(cat_naix,axis=1)
original["cat_res"]=original.apply(cat_res,axis=1)

### Primera anàlisi: Relació de naixement i residència

Aplicant les nostres categories, a primera vista es noten poques diferències entre homes i dones. Queda per investigar.

In [10]:
naix_res = original.query("sexe != '--'").\
         groupby(["cat_naix","cat_res","sexe"])["nom"].\
         count().\
         unstack("cat_res")
        

La taula mostra el percentatge entre totes les dones o tots els homes d'aquesta categoria de residència. 

Exemples:  

  - "0,958763" en primera línia, segona columna - 95.8% de les dones nascudes a la província de Barcelona resideixen a la província de Barcelona.

  - línies tres i quatre i, segona columna: entre els nascuts a Catalunya fora de la província de Barcelona, 28.2% ("0.282066") de les dones i 15.9% ("0.159961") dels homes viuen a la província de Barcelona

In [11]:

naix_res.div(naix_res.sum(axis=1),axis=0)

# naix_res

cat_res               ?         b         c         e         i
cat_naix sexe                                                  
b        Dona  0.003032  0.958763  0.036992  0.001213       NaN
         Home  0.000755  0.952946  0.042197  0.003022  0.001079
c        Dona  0.000555  0.282066  0.712382  0.004442  0.000555
         Home  0.000859  0.159961  0.835010  0.002821  0.001349
e        Dona  0.001669  0.870339  0.071786  0.055648  0.000556
         Home  0.002583  0.736810  0.143053  0.113212  0.004342
i        Dona  0.117647  0.678431  0.168627  0.023529  0.011765
         Home  0.190476  0.374474  0.391966  0.017817  0.025267

### Pas 6: executades o no?

In [12]:
def cat_execut(a) :
    if str(a).find("executa") > -1 :
        return "exec"
    else :
        return "no exec"


original["af_cat"]=original["afusellades"].apply(cat_execut)

### Primera análisi: Pena de mort i execuciones

La majoria de les 41 dones condemnades a pena de mort no van ser executades. Entre els homes, només una quarta part dels 4376 condemnats a mort no van ser executats.

In [13]:
pena_mort = original.query("pena_cat == '00 mort'").\
         groupby(["sexe","af_cat"])["pena_cat"].count().\
         unstack("sexe")
        
pena_mort_sum=pena_mort.sum(axis=0)
pena_mort_sum.name="sum"
pena_mort = pena_mort.append(pena_mort_sum)
pena_mort["sum"] = pena_mort.sum(axis=1)

pena_mort

sexe,Dona,Home,sum
af_cat,,,
exec,17,3345,3362
no exec,24,1031,1055
sum,41,4376,4417


### Pas 7: Guardar dades preparats

In [14]:
original.to_msgpack("data/processat.msg")